# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1O9aE_Hen5smf_a6TsbVF6DM0ms4Ykj6s?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
# Passo a passo

# Passo 1: Entrar na internet (abrir o navegador)

from selenium import webdriver

navegador = webdriver.Chrome()
navegador.get("https://www.google.com/")

In [2]:
# Passo 2: Importar a base de dados
import pandas as pd 

tabela = pd.read_excel("commodities.xlsx")
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Tilápia,9.05,NaN,NaN


In [3]:
# .send_keys("meu nome é lira") -> escrever nele
# .click() -> clicar nele
# .get_atribute() -> pegar uma informação dele

# import unicodedata ----------------------------------------------------\
# link = unicodedata.normalize("NFKD", link).encode("asccii", "ignore")-->formatacao de texto

# Passo 3: Para cada produto da nossa base
# Passo 4: Pegar o preço atual do produto
# Passo 5: Atualizar o preço na base de dados

for linha in tabela.index:

    produto = tabela.loc[linha, "Produto"]

    # entrar no site melhor cambio: https://www.melhorcambio.com/milho-hoje
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    link = link.replace("ó", "o").replace("ã", "a").replace("á", "a").replace(
    "ç", "c").replace("ú", "u").replace("é", "e")
    print(link)
    navegador.get(link)

    # pegar a cotação do milho
    cotacao = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    cotacao = cotacao.replace(".", "").replace(",", ".")
    cotacao = float(cotacao)
    print(cotacao)

    # na coluna Preço Atual do Milho, preencher a cotação do milho
    tabela.loc[linha, "Preço Atual"] = cotacao

https://www.melhorcambio.com/Milho-hoje
62.3
https://www.melhorcambio.com/Soja-hoje
122.53
https://www.melhorcambio.com/Boi-hoje
232.2
https://www.melhorcambio.com/Petroleo-hoje
425.22
https://www.melhorcambio.com/Algodao-hoje
411.03
https://www.melhorcambio.com/Acucar-hoje
144.06
https://www.melhorcambio.com/Cafe-hoje
1026.04
https://www.melhorcambio.com/Ouro-hoje
351.26
https://www.melhorcambio.com/Tilapia-hoje
9.5


In [4]:
# Passo 6: Decidir quais quais produtos a gente vai comprar
display(tabela)

tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,62.30,NaN
1,Soja,163.59,122.53,NaN
2,Boi,282.20,232.20,NaN
3,Petróleo,424.37,425.22,NaN
4,Algodão,497.76,411.03,NaN
5,Açúcar,136.23,144.06,NaN
6,Café,1092.87,1026.04,NaN
7,Ouro,321.77,351.26,NaN
8,Tilápia,9.05,9.50,NaN


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,62.30,True
1,Soja,163.59,122.53,True
2,Boi,282.20,232.20,True
3,Petróleo,424.37,425.22,False
4,Algodão,497.76,411.03,True
5,Açúcar,136.23,144.06,False
6,Café,1092.87,1026.04,True
7,Ouro,321.77,351.26,False
8,Tilápia,9.05,9.50,False


In [5]:
# Passo 7: Exportar a base de dados atualizada
navegador.quit()

tabela.to_excel("commodities_atualizado.xlsx", index=False)